In [4]:

#Flag, this will delete all the records for 2022 and then repopulate them from the EIA source.  The EIA source updates hourly independently.
purgeRecentRecords = True


import pandas as pd
from IPython.display import clear_output
import requests
import os

#pip install eiapy

os.environ['EIA_KEY'] = 'pGSYm7fJwZOAuwgo5jGqrOG8wsanmMFVYpaxeZSJ'
API_KEY = os.environ['EIA_KEY']

#from eiapy import Category, Series

import time

iid = 'EBA.TEX-ALL.D.H'
url = 'https://api.eia.gov/series/?api_key={}&series_id={}'.format(API_KEY, iid)

s = requests.Session()
req = s.get(url)
json_data = req.json()
#print (json_data)


## GET EIA DATA
iid = 'EBA.TEX-ALL.D.H'
#ser = Series(iid)
#print (ser.get_data(all_data=True))
#df = pd.DataFrame(ser.get_data(all_data=True)['series'][0]['data'])
df = pd.DataFrame(json_data['series'][0]['data'])
df.columns = ['dateraw', 'demand_mgwhr']
df['date'] = pd.to_datetime(df['dateraw'], format='%Y%m%dT%HZ')
df['hour_zulu'] = [x.hour for x in df['date']]
df['year'] = [x.year for x in df['date']]
del df['dateraw']
df['state'] = 'Texas'
df['dt_loaded'] = pd.Timestamp.date(pd.Timestamp.now())
df = df[['date','hour_zulu','state','demand_mgwhr','dt_loaded','year']]

print (df.head())



                 date  hour_zulu  state  demand_mgwhr   dt_loaded  year
0 2022-02-23 16:00:00         16  Texas         54584  2022-02-23  2022
1 2022-02-23 15:00:00         15  Texas         53292  2022-02-23  2022
2 2022-02-23 14:00:00         14  Texas         52011  2022-02-23  2022
3 2022-02-23 13:00:00         13  Texas         49485  2022-02-23  2022
4 2022-02-23 12:00:00         12  Texas         45418  2022-02-23  2022


In [8]:
#Insert Records

#DB Connection

import mysql.connector
from mysql.connector.constants import ClientFlag

config = {
    'user': 'demouser',
    'password': 'demopassword',
    'host': '10.86.32.2',
#    'client_flags': [ClientFlag.SSL],
#    'ssl_ca': 'ssl/server-ca.pem',
#    'ssl_cert': 'ssl/client-cert.pem',
#    'ssl_key': 'ssl/client-key.pem'
}

# now we establish our connection
cnxn = mysql.connector.connect(**config)

config['database'] = 'power'  # add new database to config dict
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

if purgeRecentRecords:
    drop_recent = "delete from power_demand where date >'2022-01-01 00:00:00'"

cursor.execute(drop_recent)

#Get Latest Record from Table to truncate what is added
latest_query=("select max(date) from power_demand")
try:
    cursor.execute(latest_query)
    last_update = cursor.fetchone()
    last_update =last_update[0].strftime('%Y-%m-%d %H:%M:%S')
    #print ("Last Updated:", last_update)
    print ("Last Record in DB:",last_update)
except:
    pass #no results in table
    last_update = '2000-01-01'
    print ("NO RECORDS in DB:",last_update)



#Insert New Records

## Trim Dataframe to new records
#print (df[df['date'] > last_update])
df = df[(df['date'] > last_update)]

print ("New Records:", len(df))


import time
df = df.sort_values(by='date', ascending=True)
for index, row in df.iterrows():
    #print(row['date'], row['hour_zulu'],row['state'],row['demand_mgwhr'],row['dt_loaded'])
    insert_query = ("INSERT INTO power_demand (date, hour_zulu, state, demand_mgwhr, dt_loaded)"
        " VALUES ('%s', '%s',' %s', '%s', '%s')" %( row['date'], row['hour_zulu'], row['state'], row['demand_mgwhr'], row['dt_loaded']))
    #print (insert_query)
    if row['year'] >= 2022:
        time.sleep(1)
        print ('Inserting Slowly...', insert_query)
    cursor.execute(insert_query)
    cnxn.commit()
    



Last Record in DB: 2022-01-01 00:00:00
New Records: 1288
Inserting Slowly... INSERT INTO power_demand (date, hour_zulu, state, demand_mgwhr, dt_loaded) VALUES ('2022-01-01 01:00:00', '1',' Texas', '46427', '2022-02-23')


KeyboardInterrupt: 